In [1]:
!python3 -m pip install --upgrade pip

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
# !pip install openai==0.27.8
!pip install openai==0.28.1

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install python-dotenv tiktoken

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
!pip install PyPDF2

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install pdf2image

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 3 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.1_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [7]:
!pip install PyMuPDF==1.23.5

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [9]:
import re
# 取得したいURL
url = "https://arxiv.org/abs/2310.03533"
identifier = re.search(r'/([^/]+)$', url).group(1)
identifier

'2310.03533'

In [10]:
import requests

def download_pdf(link, save_path):
    response = requests.get(link)
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [11]:
import os

files_to_remove = ['sample.pdf']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} を削除しました。")
    else:
        print(f"{file} は存在しません。")

sample.pdf を削除しました。


In [12]:
url = "https://arxiv.org/pdf/" + identifier

# download_pdf(url, nougat_path)
download_pdf(url, "sample.pdf")

In [13]:
pdf_path = "./sample.pdf"

In [14]:
import fitz

# PyMuPDFを使用してPDFファイルから全てのテキストを抽出
def extract_text_with_pymupdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
    return text

# テキストを抽出
text_from_pdf = extract_text_with_pymupdf(pdf_path)
def split_text_into_sections(text):
    sections = []
    current_section = None
    last_section_number = 0  # 最後に抽出したセクションの番号
    
    # セクションのタイトルと開始位置を探すための正規表現パターン
    section_pattern = re.compile(r'\n(\d+)\n([^\d\n].+?)\n', re.MULTILINE)
    
    # Abstractを探すための正規表現パターン
    abstract_pattern = re.search(r'\nabstract\n(.*?)(?=\n\d+|$)', text.lower(), re.DOTALL)
    if abstract_pattern:
        sections.append({
            'title': 'Abstract',
            'content': abstract_pattern.group(1).strip(),
            'start': abstract_pattern.start(),
            'end': abstract_pattern.end()
        })
    
    for match in section_pattern.finditer(text):
        section_number = int(match.group(1))
        section_title = match.group(2).strip()
        
        # セクション番号が前のセクション番号よりも大きいか確認
        if section_number > last_section_number:
            # "Work in progress"を含まないセクションを抽出
            if "work in progress" not in section_title.lower():
                if current_section:
                    current_section['content'] = text[current_section['end']:match.start()].strip()
                    sections.append(current_section)
                section_start = match.start()
                section_end = match.end()
                current_section = {'title': f"{section_number} {section_title}", 'content': '', 'start': section_start, 'end': section_end}
                last_section_number = section_number
        
    if current_section:
        current_section['content'] = text[current_section['end']:].strip()
        sections.append(current_section)
    
    return sections

# テキストをセクションごとに分割
sections = split_text_into_sections(text_from_pdf)

# 分割されたセクションのタイトルと最初の100文字を表示
[(section['title'], section['content'][:100]) for section in sections]


[('1 arXiv:2310.03533v3  [cs.SE]  11 Oct 2023',
  'proved advantageous in the field of Genetic Improvement [7].\nThe Automated Regression Oracle simply '),
 ('2 Section XI: Crosscutting Open Research Topic',
  'est \non\nfinal bot \nresponse\nSoftware \ndevelopment \nactivities\nRequirement Engineering\nDesign & Plann'),
 ('105 # of preprints',
  '# in CS category\n# w/ LLM in title or abstract\n# in cs.SE or cs.PL w/ LLM in title or abstract\nFig. ')]

In [15]:
import openai
import tiktoken

In [16]:
from dotenv import load_dotenv
import os

In [17]:
load_dotenv()

True

In [18]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [19]:
class ChainOfDensity:
    def __init__(self, model_name="gpt-3.5-turbo-0613"):
        self.model_name = model_name
    
    def num_tokens_from_string(self, string: str) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.encoding_for_model(self.model_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens
    
    def create_prompt(self, text):
        """Creates a prompt for the Chain of Density task."""
        return f"""
        Article: {text}

        You will generate increasingly concise, entity-dense summaries of the above Article.

        Repeat the following 2 steps 5 times.

        Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.
        Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

        A Missing Entity is:
        - Relevant: to the main story.
        - Specific: descriptive yet concise (5 words or fewer).
        - Novel: not in the previous summary.
        - Faithful: present in the Article.
        - Anywhere: located anywhere in the Article.

        Guidelines:
        - The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
        - Make every word count: re-write the previous summary to improve flow and make space for additional entities.
        - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
        - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
        - Missing entities can appear anywhere in the new summary.
        - Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. 

        Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary"
        Finally, output must be in Japanese.
        """
    
    def generate_summary(self, text):
        """Generates a summary using the Chain of Density method."""
        token_count = self.num_tokens_from_string(text)
        if token_count > 4096:
            print("Warning: Input is too long. Splitting the input into smaller chunks.")
            # 文単位で分割
            sentences = re.split(r'(?<=[.!?]) +', text)
            # 分割した文を4096トークンに収めるようにグループ化
            chunks = []
            current_chunk = ""
            for sentence in sentences:
                if len(current_chunk) + len(sentence) + 1 > 4096:
                    chunks.append(current_chunk)
                    current_chunk = sentence
                else:
                    current_chunk += (" " + sentence)
            chunks.append(current_chunk)
        else:
            chunks = [text]

        # 各チャンクに対してAPIを呼び出し
        responses = []
        for chunk in chunks:
            prompt = self.create_prompt(chunk)
            response = openai.ChatCompletion.create(
                model=self.model_name,
                messages=[
                    {"role" : "system", "content" : "Outputs should be generated in step by step."},
                    {"role": "system", "content": "Given the remaining options, what is the final answer in Japanese?"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.8,
                n=1,
                stop=None,
                top_p=1,
            )
            responses.append(response['choices'][0]['message']['content'])

        # 結果を結合して返す
        formatted_summary = " ".join(responses)
        return formatted_summary

    def generate_final_summary(self, text):
        """Generates a final summary from the combined summaries."""
        messages = [
            {"role": "system", "content": "Outputs should be generated in step by step."},
            {"role": "system", "content": "Given the remaining options, what is the final answer in Japanese?"},
            {"role": "system", "content": "Please summarize all of the following sentences, no matter how long they may be, including all of the following information."},
            {"role": "user", "content": text}
        ]
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=messages,
            temperature=0.8,
            n=1,
            stop=None,
            top_p=1,
        )
        return response['choices'][0]['message']['content']
    
    def format_cod_output(self, cod_output):
        """Formats the output of Chain of Density to be a readable text."""
        try:
            summaries = json.loads(cod_output)
            formatted_summaries = " ".join([summary['Denser_Summary'] for summary in summaries])
            return formatted_summaries
        except json.JSONDecodeError:
            print("Error: The output is not in valid JSON format.")
            return None

In [20]:
# 論文タイトル
with fitz.open(pdf_path) as doc:
    page_num = 0
    page = doc.load_page(page_num)
    text = page.get_text()

# Extract text until "Abstract" or "ABSTRACT"
abstract_index = text.lower().find("abstract")
pdf_title = text[:abstract_index ] if abstract_index != -1 else "Abstract not found"
print(pdf_title)

Large Language Models for Software Engineering:
Survey and Open Problems
Angela Fan
Generative AI Team
Meta Platforms Inc.
New York, NY, USA
Beliz Gokkaya
PyTorch Team
Meta Platforms Inc.
Menlo Park, CA, USA
Mark Harman
Instagram Product Foundation
Meta Platforms Inc.
London, UK
Mitya Lyubarskiy
Developer Infrastructure
Meta Platforms Inc.
London, UK
Shubho Sengupta
FAIR
Meta Platforms Inc.
Menlo Park, CA, USA
Shin Yoo
School of Computing
KAIST
Daejeon, Korea
Jie M. Zhang
Department of Informatics
King’s College London
London, UK



In [21]:
sections[0]

{'title': '1 arXiv:2310.03533v3  [cs.SE]  11 Oct 2023',
 'content': 'proved advantageous in the field of Genetic Improvement [7].\nThe Automated Regression Oracle simply uses the existing\nversion of the software system as a reference against which to\nbenchmark output from any subsequent adaptions and changes.\nOf course, there is a risk of ‘baking in’ functional incor-\nrectness, since the Automated Regression Oracle cannot detect\nwhat the system should do, but only capture what it currently\ndoes. Therefore, the Automated Regression Oracle can test\nonly for functional regressions so it is best suited to use cases\nwhere the existing functionality is to be maintained. For ex-\nample, for non-functional improvements such as performance\noptimisation and for semantics-preserving refactoring.\nThe input provided to an LLM will be a natural focus of\ngrowing research, and we can expect a rapid development of\nthe literature on prompt engineering and prompt optimisation\n[8]. In this su

In [22]:
cod = ChainOfDensity()
abst_summary = cod.generate_summary(sections[0]['content'])
print(abst_summary)

{"Missing_Entities": "Automated Regression Oracle; functional regressions", "Denser_Summary": "既存のソフトウェアシステムを基準にして、自動回帰オラクルはその後の適応と変更の出力を評価する。機能の再現テストしかできないため、既存の機能を維持する場合に最適。パフォーマンスの最適化や意味を保持するリファクタリングには適している。"}


In [24]:
# CoDのインスタンスを作成
cod = ChainOfDensity()
# 各章に対してIEPを適用
for section in sections[:-1]:
    with Timer(prefix=f'"Chapter: {section["title"]}'):
        # 章ごとの前提とする要約文章の生成
        with Timer(prefix=f'"generate_summary: '):
            formatted_summary = cod.generate_summary(section['content']).split("\n")
            # print(f"{formatted_summary}")
            print("\n" + "-"*50 + "\n")
        # 最終回答
        with Timer(prefix=f'"Final Answer: '):
            final_answer = cod.generate_final_summary(str(formatted_summary))
            print(f"{final_answer}")
    print("\n" + "="*50 + "\n")


--------------------------------------------------

"generate_summary:  8.967[s]
自動回帰オラクルは、既存のソフトウェアシステムを参照として使用し、後続の適応と変更の出力をベンチマークにする。しかし、自動回帰オラクルはシステムが行うべきことを検出することはできず、現在の動作のみを捉えるため、機能的な不正確さを「焼き込む」リスクがある。そのため、自動回帰オラクルは機能の回帰テストのみを行うことができるため、既存の機能を維持する場合に最適である。例えば、パフォーマンスの最適化や意味を保持するリファクタリングなどの非機能的な改善に適している。プロンプトエンジニアリングとプロンプト最適化に関する既存の研究や課題を強調する本調査では、LLMへの入力は成長する研究の自然な焦点となる可能性があり、LLMの出力はコードに限定される必要はない。要件、テストケース、設計図、ドキュメンテーションなど、ソフトウェアエンジニアリングの他のアーティファクトも含めることができる。また、LLMの主な出力とともに提供される説明も重要な出力である。
"Final Answer:  8.700[s]
"Chapter: 1 arXiv:2310.03533v3  [cs.SE]  11 Oct 2023 17.667[s]



--------------------------------------------------

"generate_summary:  16.595[s]
この記事では、ソフトウェア開発活動について議論されています。要件エンジニアリングや設計、コードの実装、テスト、保守、展開など、さまざまなフェーズがあります。さらに、ソフトウェアのテストやパフォーマンス改善、デバッグと修正、文書生成、新たなテストの生成など、さまざまな技術や手法が研究されています。

この記事では、ソフトウェアの応答に関する情報や詳細も議論されています。

また、要件エンジニアリングや設計、計画立案に関する情報や詳細も議論されています。

さらに、コードの実装やテストに関する情報や詳細も議論されています。
"Final Answer:  5.149[s]
"Chapter: 2 Section XI: Crosscutti

画像の抽出

In [ ]:
!pip install fitz

In [ ]:
!pip uninstall -y PyMuPDF
!pip install PyMuPDF

In [ ]:
!pip install fitz

In [ ]:
import fitz  # PyMuPDF
from PIL import Image

# pdf_path = nougat_path
pdf_path = "sample.pdf"
doc = fitz.open(pdf_path)
extracted_images = []

for page in doc:
    # ページからイメージを抽出
    for img_index in page.get_images(full=True):
        xref = img_index[0]
        base_image = fitz.Pixmap(doc, xref)
        pil_img = Image.frombytes("RGB", [base_image.width, base_image.height], base_image.samples)
        extracted_images.append(pil_img)

# 最初の抽出された図を表示
if extracted_images:
    extracted_images[0].show()
else:
    "このPDFには抽出可能な図が含まれていません。"


In [ ]:
for img in extracted_images:
    img.show()